<a href="https://colab.research.google.com/github/AI-Cultural-Heritage-Lab/llm_sanitization/blob/main/USHMM_Combined_Data_Ulysses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Combine Data into 1 notebook

In [ ]:
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default
import re

# Authenticate
auth.authenticate_user()
credentials, _ = default()
gc = gspread.authorize(credentials)

# List of Google Sheet URLs
sheet_urls = [
    "https://drive.google.com/open?id=18debecssSz0FIdrTbDrFT075-ekXwJp6lHwe-5UPZug&usp=drive_copy",
    "https://drive.google.com/open?id=1qIkwpLSOZqD6GNXEXxtRW77ptzeKlPty4hBWs7haKr8&usp=drive_copy",
    "https://drive.google.com/open?id=1o8S1-KFS7K0Q9KeF9zKdrDRACQPsI0Db9qVuW1GyurE&usp=drive_copy",
    "https://drive.google.com/open?id=1f4Sl6chwo9Wf7AQlZc9xjRtBonErQd3s5zzZBjeLkSQ&usp=drive_copy",
    "https://drive.google.com/open?id=1CJjZ3QOUSduDOO4s3m7jllCuSPf50J3Q7ArDO71KkcU&usp=drive_copy",
    "https://drive.google.com/open?id=19O8XSv8ZO4jn725Ww6Pu8OojbhfhCvDArVV0iSyagnM&usp=drive_copy"
]

# Extract sheet IDs from URLs
sheet_ids = []
for url in sheet_urls:
    # Extract ID from URL
    match = re.search(r'id=([^&]+)', url)
    if match:
        sheet_ids.append(match.group(1))

# Read and process all sheets
dataframes = []
for sheet_id in sheet_ids:
    try:
        # Open the spreadsheet using the ID
        spreadsheet = gc.open_by_key(sheet_id)

        # Get spreadsheet title for file name
        sheet_name = spreadsheet.title

        # Get the first worksheet
        worksheet = spreadsheet.sheet1

        # Get all values
        data = worksheet.get_all_records()

        # Convert to DataFrame
        df = pd.DataFrame(data)

        # Add file name and index
        df['file_name'] = sheet_name
        df['file_row_index'] = df.index

        dataframes.append(df)
        print(f"Successfully processed: {sheet_name}")

    except Exception as e:
        print(f"Error processing sheet {sheet_id}: {e}")

# Concatenate all DataFrames
if dataframes:
    combined_df = pd.concat(dataframes, ignore_index=True)

    # Reorder columns to place 'file_name' and 'file_row_index' at the front
    combined_df = combined_df[['file_name', 'file_row_index'] +
                 [col for col in combined_df.columns if col not in ['file_name', 'file_row_index']]]

    # Display the resulting DataFrame structure
    print(f"\nCombined {len(dataframes)} sheets into one DataFrame with {len(combined_df)} rows.")
    display(combined_df)
else:
    print("No valid Google Sheets processed.")

Successfully processed: Copy of Holocaust Encyclopedia - search queries from United States 2024
Successfully processed: Copy of Holocaust Encyclopedia - search queries from Germany 2024
Successfully processed: Copy of Holocaust Encyclopedia - search queries from South Korea 2024
Successfully processed: Copy of Holocaust Encyclopedia - search queries from Russia 2024
Successfully processed: Copy of Holocaust Encyclopedia - search queries from Turkey 2024
Successfully processed: Copy of Holocaust Encyclopedia - search queries from China 2024

Combined 6 sheets into one DataFrame with 6000 rows.


,file_name,file_row_index,Top queries,Clicks,Impressions,CTR,Position
0,Copy of Holocaust Encyclopedia - search querie...,0,how many people died in the holocaust,40678,189593,21.46%,1.02
1,Copy of Holocaust Encyclopedia - search querie...,1,armenian genocide,37535,447164,8.39%,2.44
2,Copy of Holocaust Encyclopedia - search querie...,2,holocaust encyclopedia,35664,50277,70.94%,1.00
3,Copy of Holocaust Encyclopedia - search querie...,3,first they came for,34702,77617,44.71%,1.08
4,Copy of Holocaust Encyclopedia - search querie...,4,holocaust,33996,1330137,2.56%,1.91
...,...,...,...,...,...,...,...
5995,Copy of Holocaust Encyclopedia - search querie...,995,欧洲战争,1,7,14.29%,8.00
5996,Copy of Holocaust Encyclopedia - search querie...,996,戈德曼审判,1,7,14.29%,8.43
5997,Copy of Holocaust Encyclopedia - search querie...,997,人民冲锋队,1,7,14.29%,9.14
5998,Copy of Holocaust Encyclopedia - search querie...,998,犹太人在乌克兰,1,7,14.29%,9.57


In [ ]:
import re

# After creating the combined DataFrame but before saving to CSV:

# Function to extract location and year from title
def extract_location_year(title):
    # Pattern to match "from [Location] [Year]" at the end of the string
    pattern = r'from\s+([A-Za-z\s]+)\s+(\d{4})$'
    match = re.search(pattern, title)

    if match:
        location = match.group(1).strip()
        year = match.group(2)
        return location, year
    else:
        return None, None

# Create new columns for location and year
combined_df['location'] = None
combined_df['year'] = None

# Extract location and year from file_name
for idx, row in combined_df.iterrows():
    location, year = extract_location_year(row['file_name'])
    combined_df.at[idx, 'location'] = location
    combined_df.at[idx, 'year'] = year

# Move the new columns after file_name and file_row_index
column_order = ['file_name', 'file_row_index', 'location', 'year'] + [
    col for col in combined_df.columns
    if col not in ['file_name', 'file_row_index', 'location', 'year']
]
combined_df = combined_df[column_order]

# Display the result to verify
display(combined_df.head())

,file_name,file_row_index,location,year,Top queries,Clicks,Impressions,CTR,Position
0,Copy of Holocaust Encyclopedia - search querie...,0,United States,2024,how many people died in the holocaust,40678,189593,21.46%,1.02
1,Copy of Holocaust Encyclopedia - search querie...,1,United States,2024,armenian genocide,37535,447164,8.39%,2.44
2,Copy of Holocaust Encyclopedia - search querie...,2,United States,2024,holocaust encyclopedia,35664,50277,70.94%,1.00
3,Copy of Holocaust Encyclopedia - search querie...,3,United States,2024,first they came for,34702,77617,44.71%,1.08
4,Copy of Holocaust Encyclopedia - search querie...,4,United States,2024,holocaust,33996,1330137,2.56%,1.91


In [ ]:
# After creating combined_df as shown in the previous code
import os

# Set the path where you want to save the CSV
output_folder = '/content/drive/MyDrive/AI and Cultural Heritage Lab/Current Projects/LLM Language Sanitization Project/Datasets/'
output_file = os.path.join(output_folder, 'combined_data.csv')

# Save the DataFrame to CSV
try:
    combined_df.to_csv(output_file, index=False)
    print(f"Successfully saved combined data to: {output_file}")
except Exception as e:
    print(f"Error saving CSV file: {e}")


Successfully saved combined data to: /content/drive/MyDrive/AI and Cultural Heritage Lab/Current Projects/LLM Language Sanitization Project/Datasets/combined_data.csv


In [ ]:
# After creating combined_df with all your data:

# Define the Google Sheet ID from your URL
target_sheet_id = '1_qAn-BNXdIzotr_bXXF2dAjJfa7pDFH1Q84Y9fEBby4'
gid = '1901843023'  # Optional: only needed if you want to target a specific sheet tab

try:
    # Open the target spreadsheet
    target_spreadsheet = gc.open_by_key(target_sheet_id)

    # If you know the specific worksheet name, use this:
    # worksheet = target_spreadsheet.worksheet("Sheet1")

    # Or target by gid (worksheet index):
    # Find worksheet with specific gid
    for worksheet in target_spreadsheet.worksheets():
        if worksheet.id == gid:
            target_worksheet = worksheet
            break
    else:
        # If no matching worksheet found, use the first one
        target_worksheet = target_spreadsheet.sheet1

    # Clear the existing content
    target_worksheet.clear()

    # Convert DataFrame to list of lists (including header)
    data_to_upload = [combined_df.columns.tolist()] + combined_df.values.tolist()

    # Update the sheet with all values at once
    target_worksheet.update(data_to_upload)

    print(f"Successfully updated Google Sheet: {target_spreadsheet.title}")

except Exception as e:
    print(f"Error updating Google Sheet: {e}")

Successfully updated Google Sheet: combined_data
